# Demo Program WCF

In [8]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate
import surprise.accuracy

# Import Data User, Anime, UserAnime

In [9]:
folder = "../Dataset SKRIPSI/"

filename = folder + "users_data"
UsersDF = pickle.load(open(filename,'rb'))

filename = folder + "animes_data"
AnimesDF = pickle.load(open(filename,'rb'))

filename = folder + "scores_data"
ScoresDF = pickle.load(open(filename,'rb'))
CFScoresDF = pickle.load(open(filename,'rb'))

# Membobotkan Nilai Status

In [10]:
conditions = [
    (ScoresDF['my_status'] == 2),
    (ScoresDF['my_status'] == 3),
    (ScoresDF['my_status'] == 6),
    (ScoresDF['my_status'] == 1),
    (ScoresDF['my_status'] == 4)
]

values = [5,2,2,3,1]

ScoresDF['status_bobot'] = np.select(conditions,values)

In [11]:
ScoresDF.loc[ScoresDF['my_status'] == None]

,username,anime_id,my_score,my_status,animes_rated,status_bobot


In [12]:
ScoresDF['scoreXstatus'] = ScoresDF['my_score'] * ScoresDF['status_bobot']
ScoresDF['scoreXstatus'] = ScoresDF['scoreXstatus'] / 5
ScoresDF

,username,anime_id,my_score,my_status,animes_rated,status_bobot,scoreXstatus
0,Tsundora,21,2,4,259,1,0.4
1,Tsundora,59,5,2,259,5,5.0
2,Tsundora,210,5,2,259,5,5.0
3,Tsundora,249,7,4,259,1,1.4
4,Tsundora,269,6,2,259,5,6.0
...,...,...,...,...,...,...,...
193267,Senpai_Helix,4214,10,2,4,5,10.0
193268,Senpai_Helix,18677,9,2,4,5,9.0
193269,Kiyoshi-Kun,1722,10,2,3,5,10.0
193270,Kiyoshi-Kun,2993,10,2,3,5,10.0


# Membuat User-Item Matriks Berdasarkan Rating
Serta mengisi data yang kosong dengan 0

In [13]:
#WCF
UserItemRatingMatrix = ScoresDF.pivot(index='username',columns='anime_id',values='scoreXstatus')
UserItemRatingMatrix = UserItemRatingMatrix.replace(np.nan,0)
UserItemRatingMatrix

anime_id,1,5,6,7,8,15,16,17,18,19,...,37277,37283,37320,37324,37341,37408,37433,37505,37672,37719
username,,,,,,,,,,,,,,,,,,,,,
-Axel-,10.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Belatriz-,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-INFERNO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Jhon-,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Sato-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zerorider,10.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ziomalos,9.0,8.0,9.0,7.0,0.0,8.0,9.0,8.0,8.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zipyto,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
#CF
CFUserItemRatingMatrix = ScoresDF.pivot(index='username',columns='anime_id',values='my_score')
CFUserItemRatingMatrix = CFUserItemRatingMatrix.replace(np.nan,0)
CFUserItemRatingMatrix

anime_id,1,5,6,7,8,15,16,17,18,19,...,37277,37283,37320,37324,37341,37408,37433,37505,37672,37719
username,,,,,,,,,,,,,,,,,,,,,
-Axel-,10.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Belatriz-,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-INFERNO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Jhon-,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Sato-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zerorider,10.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ziomalos,9.0,8.0,9.0,7.0,0.0,8.0,9.0,8.0,8.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zipyto,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Training dan Test Model WCF
Training dan test model dipisah 80% : 20%

In [15]:
reader = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(ScoresDF[['username','anime_id','scoreXstatus']],reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=50)

In [16]:
sim_options = {'name': 'pearson',
                'min_support': 1
               }
algo_knn = KNNBasic(k=40, sim_options=sim_options)
prediction_knn = algo_knn.fit(trainset).test(testset)

# Prediksi
prediction_knn

Computing the pearson similarity matrix...


d:\Python\lib\site-packages\surprise\prediction_algorithms\algo_base.py:249: RuntimeWarning: invalid value encountered in sqrt
  sim = construction_func[name](*args)


Done computing similarity matrix.


[Prediction(uid='Shouichirou', iid=12679, r_ui=8.0, est=7.202972926368368, details={'actual_k': 32, 'was_impossible': False}),
 Prediction(uid='aleigh', iid=2904, r_ui=9.0, est=9.305482989237236, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='bigkfc2k8', iid=267, r_ui=6.0, est=7.707358535472425, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='Torah', iid=6956, r_ui=10.0, est=7.700171719365383, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='godT', iid=63, r_ui=7.0, est=6.522648451937305, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='lightofzeref', iid=36511, r_ui=4.2, est=4.0205377441373775, details={'actual_k': 14, 'was_impossible': False}),
 Prediction(uid='ranmaSA', iid=401, r_ui=10.0, est=8.280047126783222, details={'actual_k': 33, 'was_impossible': False}),
 Prediction(uid='Josay', iid=16664, r_ui=9.0, est=8.361478435739782, details={'actual_k': 32, 'was_impossible': False}),
 Prediction(uid='Va

# Training dan Test Model CF
Training dan test model dipisah 80% : 20%

In [17]:
reader = Reader(rating_scale=(1,10))
CFdata = Dataset.load_from_df(CFScoresDF[['username','anime_id','my_score']],reader)

CFtrainset, CFtestset = train_test_split(CFdata, test_size=0.2, random_state=50)

In [18]:
sim_options = {'name': 'pearson',
                'min_support': 1
               }
CFalgo_knn = KNNBasic(k=40, sim_options=sim_options)
CFprediction_knn = CFalgo_knn.fit(CFtrainset).test(CFtestset)

# Prediksi
CFprediction_knn

Computing the pearson similarity matrix...
Done computing similarity matrix.


[Prediction(uid='Shouichirou', iid=12679, r_ui=8.0, est=7.298508737317011, details={'actual_k': 34, 'was_impossible': False}),
 Prediction(uid='aleigh', iid=2904, r_ui=9.0, est=9.31526811825633, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='bigkfc2k8', iid=267, r_ui=6.0, est=7.713063281509377, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='Torah', iid=6956, r_ui=10.0, est=7.7678016961654155, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='godT', iid=63, r_ui=7.0, est=7.329945525914593, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='lightofzeref', iid=36511, r_ui=7.0, est=6.593916355103369, details={'actual_k': 14, 'was_impossible': False}),
 Prediction(uid='ranmaSA', iid=401, r_ui=10.0, est=8.623912714700671, details={'actual_k': 29, 'was_impossible': False}),
 Prediction(uid='Josay', iid=16664, r_ui=9.0, est=8.309503488388447, details={'actual_k': 32, 'was_impossible': False}),
 Prediction(uid='Van

# Mendapatkan List Rekomendasi

In [23]:
username = "aleigh"
# get the list of the movie ids
unique_ids = ScoresDF['anime_id'].unique()
id_rec = ScoresDF.loc[ScoresDF['username']==username, 'anime_id']
anime_to_pred = np.setdiff1d(unique_ids, id_rec)

In [24]:
list_rec = []

for iid in anime_to_pred:
    list_rec.append((iid, algo_knn.predict(uid=username, iid=iid).est))

aleigh = pd.DataFrame(list_rec, columns=['iid', 'predict']).sort_values('predict', ascending=False)

In [28]:
CFlist_rec = []

for iid in anime_to_pred:
    CFlist_rec.append((iid, CFalgo_knn.predict(uid=username, iid=iid).est))

CFaleigh = pd.DataFrame(CFlist_rec, columns=['iid', 'predict']).sort_values('predict', ascending=False)

# Prediksi

In [26]:
surprise.accuracy.mae(prediction_knn)
surprise.accuracy.rmse(prediction_knn)

MAE:  1.4775
RMSE: 1.9834


1.9834231529616737

In [27]:
surprise.accuracy.mae(CFprediction_knn)
surprise.accuracy.rmse(CFprediction_knn)

MAE:  1.2023
RMSE: 1.5561


1.5560680417364003